# TWI Calculator — Google Earth Engine + Colab

Compute the **Topographic Wetness Index (TWI)** for a user-selected area in an interactive workflow.  
TWI is defined as `ln(a / tan β)`, where *a* is upslope contributing area per unit contour and *β* is slope (radians).

## What this notebook does
- **Sign in & project setup:** Authenticate with Earth Engine and set your Cloud Project ID.
- **AOI selection:** Draw a polygon/rectangle. Optional: define a buffer. The buffer is used **for slope and flow accumulation** to reduce edge effects; results are **clipped back** to the original AOI.
- **DEM sources (selectable):**  
  • FABDEM — 30 m, DTM (forests & buildings removed)  
  • Copernicus GLO-30 — 30 m, DSM  
  • ALOS World-3D (AW3D30) — 30 m, DSM  
  • NASA SRTM — 30 m, DSM  
  • NASADEM — 30 m, DSM  
  • ASTER GDEM — 30 m, DSM  
  • CGIAR SRTM-90 — 90 m, DSM  
  • MERIT Hydro — 90 m, DTM, hydrologically adjusted  
- **Hydrologic corrections:** Depression filling and flat resolution
- **Flow routing (selectable):**
  - **MFD** (Quinn 1991)
  - **D8**
- **Outputs:** Flow accumulation (km²), slope, and **TWI**.  
  Optional reference layer: **CTI** from Hydrography90m (values unscaled by `1e8`).
- **Export:** One-click export of TWI (and other layers) as GeoTIFF — either to Google Drive (if billing is active) or directly to local storage.

*Notes:* TWI are proxies that depend on DEM quality and routing choice; interpret results accordingly.


<details open>
<summary><h2>📘 Instructions</h2></summary>

<h3>Running the Notebook</h3>

To run this notebook, you must first <b>save a copy to your own Google account</b> (the one registered with Google Earth Engine):  
<b>File → Save a copy in Drive.</b>

If another user needs to sign in to Colab, several steps must be completed in <b>Google Cloud Console:</b>  
<a href="https://console.cloud.google.com/">https://console.cloud.google.com/</a>


<h3>1) Account Verification and Project Registration in Google Earth Engine</h3>

The user must be signed in to their <b>Google account</b>, which contains at least one <b>Google Cloud project</b> registered for either <b>commercial or non-commercial use in Google Earth Engine (GEE)</b>.

<b>Steps to verify or register your project:</b>
<ol>
<li>Open <b>Google Cloud Console</b>.</li>
<li>In the left navigation menu, select <b>View all products</b>.</li>
<li>Under the <b>Analytics</b> section, click <b>Earth Engine → Configuration</b>.</li>
<li>
If your project is already registered, you will see the message:<br>
<i>“Your Cloud project is registered for non-commercial (or commerscial) use.”</i><br>
If not, under <i>“Register your Cloud project”</i> click <i>“Register”</i>, then choose registration for either <b>commercial</b> or <b>non-commercial</b> use and complete the short form.

</li>
<li>After completing registration, copy your <b>project ID</b> and paste it into the appropriate cell in Colab.  
You can find your project ID, for example, in <b>Navigation menu → IAM & Admin → Settings</b>.</li>
</ol>


<h3>2) Enabling Required APIs</h3>

In <b>Google Cloud Console</b>, you must enable two APIs that Colab uses when working with GEE:

<ol>
<li>Open <b>Google Cloud Console</b>.</li>
<li>Ensure the correct project is selected (the same one registered in GEE).</li>
<li>In the top search bar, look up:
<ul>
<li><b>Google Earth Engine API</b></li>
<li><b>Cloud Billing API</b></li>
</ul>
</li>
<li>Click <b>Enable</b> for both APIs to activate them for your project.</li>
</ol>


<h3>3) Creating a Service Account</h3>

Next, you need to create a <b>Service Account</b> that Colab will use to authenticate access to Google Earth Engine.

<b>Steps:</b>
<ol>
<li>In <b>Google Cloud Console</b>, go to <b>Navigation menu → IAM & Admin → Service Accounts</b>.</li>
<li>Click <b>Create service account</b>.</li>
<li>Enter any name (e.g., <code>gee-colab-access</code>) and click <b>Create and continue</b>.</li>
<li>Under <b>Permissions</b>, assign the role <b>Editor</b>.</li>
<li>Click <b>Done</b>.</li>
</ol>


<h3>4) Obtaining the Calculation Results</h3>

The output file can be obtained in two ways:

<h4>a) Save to Google Drive</h4>

This option requires an <b>active Billing Account</b>.

<ol>
<li>Open <b>Navigation menu → Billing</b> and check whether you have an active billing account.</li>
<li>If none exists, create one via <b>Navigation menu → Billing → Create billing account</b>.
<ul>
<li>Enter an account name.</li>
<li>Add a valid payment card.</li>
</ul>
</li>
<li>Link your project to this billing account:<br>
<b>Navigation menu → IAM & Admin → Settings → Manage billing → Link a billing account.</b></li>
</ol>

<h4>b) Download to Local Storage</h4>

Alternatively, you can download the <b>TWI result</b> directly in <b>GeoTIFF</b> format to your local computer.

</details>

In [1]:
#@title 🔧 Environment setup (clone repository & install dependencies)

#@markdown ### 🔹 **Before proceeding, run this cell.**
#@markdown It prepares the working environment for all subsequent steps.

#@markdown ---
#@markdown ### 🔹 **What this cell does:**
#@markdown 1) Clones the **gee_twi** GitHub repository into this Colab workspace.
#@markdown 2) Installs all required Python packages (Earth Engine API, Google Cloud Billing client, geemap, rasterio, leafmap, localtileserver).
#@markdown 3) Imports and initializes core libraries for Earth Engine and local raster processing.
#@markdown 4) Enables the Colab custom widget manager so interactive maps render correctly.
!git clone https://github.com/barysvla/gee_twi.git

# Install required packages
%pip install -q earthengine-api google-cloud-billing geemap rasterio leafmap localtileserver --quiet
import ee, geemap, numpy, rasterio, tempfile, os, ipywidgets as widgets

# Enable support for interactive Jupyter widgets (e.g., maps, sliders, GUIs) in Google Colab.
# This allows libraries like leafmap, geemap, and ipywidgets to render properly inside Colab notebooks.
from google.colab import output
output.enable_custom_widget_manager()

%cd gee_twi

Cloning into 'gee_twi'...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 278 (delta 64), reused 5 (delta 5), pack-reused 164 (from 1)
Receiving objects: 100% (278/278), 146.06 KiB | 1.15 MiB/s, done.
Resolving deltas: 100% (111/111), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.9/667.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [3]:
#@title 🌍 Earth Engine Sign-In & Project Initialization { display-mode: "form" }

#@markdown ### 🔹 **Enter your Cloud Project ID**
#@markdown Provide the **Google Cloud Project ID** that is registered in **Google Earth Engine**.
project_id = "twi-test-472912"  #@param {type:"string"}

#@markdown ---
#@markdown ### 🔹 **What happens next**
#@markdown After you run this cell:
#@markdown 1. You will be authenticated and Earth Engine will be initialized with your project.
#@markdown 2. The script checks whether **Cloud Billing is enabled** for this project.
#@markdown 3. You can choose the execution mode:
#@markdown    - **Auto (recommended):** Cloud if billing enabled, otherwise Local
#@markdown    - **Cloud:** only if billing is enabled
#@markdown    - **Local:** always available

#@markdown ▶ **Important:** When signing in, make sure to **allow all pop-up requests** and **confirm every prompt** to complete authentication successfully.

import os
import ee
import ipywidgets as widgets
from IPython.display import display

# ------------------------------------------------------------
# 1) Validate input project id
# ------------------------------------------------------------
if not project_id.strip():
    raise ValueError("Please enter your Earth Engine Cloud Project ID (e.g., 'my-ee-project').")

GEE_PROJECT_ID = project_id.strip()

# ------------------------------------------------------------
# 2) Authenticate + initialize Earth Engine
# ------------------------------------------------------------
try:
    ee.Initialize(project=GEE_PROJECT_ID)
    print(f"✅ Earth Engine initialized with project: {GEE_PROJECT_ID}")
except Exception:
    # Colab auth + EE auth fallback
    from google.colab import auth as colab_auth
    colab_auth.authenticate_user()
    ee.Authenticate()
    ee.Initialize(project=GEE_PROJECT_ID)
    print(f"✅ Earth Engine initialized with project: {GEE_PROJECT_ID}")

# Persist project id for later cells
os.environ["EE_PROJECT_ID"] = GEE_PROJECT_ID

# ------------------------------------------------------------
# 3) Billing check via Cloud Billing API
# ------------------------------------------------------------
billing_exists = False          # whether user can see at least one billing account
project_linked = False          # whether THIS project has billing enabled
linked_account_name = None      # resource name of the linked billing account (if any)
error_msg = None

try:
    from google.cloud import billing_v1
    from google.api_core import exceptions as gexc

    client = billing_v1.CloudBillingClient()

    # 3.1) Check if any billing accounts are visible (permission may be restricted)
    try:
        for _ in client.list_billing_accounts():
            billing_exists = True
            break
    except gexc.PermissionDenied:
        # Not critical; user may not have permission to list accounts
        pass

    # 3.2) Check if this project has billing enabled
    try:
        info = client.get_project_billing_info(name=f"projects/{GEE_PROJECT_ID}")
        project_linked = bool(getattr(info, "billing_enabled", False))
        linked_account_name = getattr(info, "billing_account_name", None) or None
    except gexc.NotFound as e:
        error_msg = f"Project not found in Cloud Billing API: {e.message}"
    except gexc.PermissionDenied as e:
        error_msg = f"No permission to read project billing info: {e.message}"
    except Exception as e:
        error_msg = f"Unexpected billing check error: {e}"

except ImportError:
    error_msg = "google-cloud-billing is not installed."

print("🔎 Any billing accounts visible to you:", billing_exists)
print("🔎 Project billing enabled:", project_linked)
print("🔎 Linked billing account:", linked_account_name)
if error_msg:
    print("⚠️ Billing check note:", error_msg)

# ------------------------------------------------------------
# 4) Mode selection UI (Auto / Cloud / Local)
# ------------------------------------------------------------
# We keep both variables:
# - PIPELINE_MODE: "cloud" or "local" (readable)
# - USE_BUCKET: bool for compatibility with existing code (True -> cloud)
PIPELINE_MODE = None
USE_BUCKET = None

# UI widgets
mode_dd = widgets.Dropdown(
    options=[
        ("Auto (recommended)", "auto"),
        ("Cloud (Earth Engine)", "cloud"),
        ("Local (Colab)", "local"),
    ],
    value="auto",
    description="Mode:",
    style={"description_width": "70px"},
    layout=widgets.Layout(width="420px", margin="0 0 0 -10px"),
)

apply_btn = widgets.Button(
    description="Apply mode",
    button_style="primary",
    layout=widgets.Layout(margin="6px 0 0 0"),
)

mode_out = widgets.Output()

def _resolve_mode(choice: str, billing_ok: bool) -> str:
    """Resolve requested mode to an actual allowed mode."""
    if choice == "auto":
        return "cloud" if billing_ok else "local"
    if choice == "cloud":
        return "cloud" if billing_ok else "local"
    return "local"

@apply_btn.on_click
def _apply_mode(_):
    global PIPELINE_MODE, USE_BUCKET
    mode_out.clear_output()

    requested = mode_dd.value
    resolved = _resolve_mode(requested, project_linked)

    # Enforce constraints: no cloud if billing is not enabled
    PIPELINE_MODE = resolved
    USE_BUCKET = (resolved == "cloud")

    with mode_out:
        print(f"Requested: {requested}")
        print(f"Resolved:  {resolved}")
        if requested == "cloud" and not project_linked:
            print("⚠️ Cloud mode is not available because billing is not enabled for this project. Using Local mode.")
        if requested == "auto":
            if project_linked:
                print("✅ Auto selected Cloud mode (billing enabled).")
            else:
                print("✅ Auto selected Local mode (billing not enabled).")
        print(f"USE_BUCKET = {USE_BUCKET}")

# Disable the Cloud option visually if billing is not enabled
# (Dropdown itself cannot disable a single option, but we can warn + resolve)
ui = widgets.VBox([mode_dd, apply_btn, mode_out])
display(ui)

# ------------------------------------------------------------
# 5) Set default automatically (no click needed)
# ------------------------------------------------------------
# If you want the cell to immediately set the best mode, do it here:
PIPELINE_MODE = _resolve_mode("auto", project_linked)
USE_BUCKET = (PIPELINE_MODE == "cloud")
print(f"⚙️ Default mode set automatically: {PIPELINE_MODE} (USE_BUCKET={USE_BUCKET})")
print("You can change it in the dropdown and click 'Apply mode'.")

✅ Earth Engine initialized with project: twi-test-472912
🔎 Any billing accounts visible to you: True
🔎 Project billing enabled: True
🔎 Linked billing account: billingAccounts/01698C-8AB00A-10CC20


⚙️ Default mode set automatically: cloud (USE_BUCKET=True)
You can change it in the dropdown and click 'Apply mode'.


In [7]:
#@title 🗺️ Select AOI (Draw on Map OR Upload Vector File) { display-mode: "form" }
#@markdown ### Define AOI in one of two ways:
#@markdown 1) **Draw** a polygon/rectangle on the map, or
#@markdown 2) **Upload** a vector file (`.geojson/.json`, `.gpkg`, `.kml/.kmz`, or **zipped Shapefile** `.zip`)
#@markdown
#@markdown The selected AOI is stored as `AOI_EE` (ee.Geometry) and will be used in the pipeline cell.

import os
import zipfile
import geopandas as gpd
import ee
import geemap
import ipywidgets as widgets
from google.colab import files
from IPython.display import display

# --- Map init (same as you already have) ---
Map = geemap.Map(basemap="Esri.WorldImagery")
Map.add_basemap("Esri.WorldTopoMap")
display(Map)

# --- UI controls ---
mode_toggle = widgets.ToggleButtons(
    options=[("Draw on map", "draw"), ("Upload file", "upload")],
    value="draw",
    description="AOI mode:",
    style={"description_width": "90px"},
)

upload_btn = widgets.Button(description="Upload AOI file", button_style="primary")
status = widgets.Output()

display(widgets.VBox([mode_toggle, upload_btn, status]))

# Global AOI holder (ee.Geometry)
AOI_EE = None

def _gdf_to_ee_geometry(gdf: gpd.GeoDataFrame) -> ee.Geometry:
    """
    Convert GeoDataFrame to a single ee.Geometry (union of all features).
    Uses WGS84 to avoid CRS issues.
    """
    # Ensure geometry exists
    gdf = gdf[gdf.geometry.notnull()].copy()
    if len(gdf) == 0:
        raise ValueError("No valid geometries found in the uploaded file.")

    # Reproject to WGS84 (EPSG:4326) for reliable EE conversion
    if gdf.crs is None:
        # If missing CRS, treat coordinates as WGS84 (this is the only sane default)
        gdf = gdf.set_crs(epsg=4326)
    else:
        gdf = gdf.to_crs(epsg=4326)

    # Dissolve all geometries into one (AOI)
    geom = gdf.unary_union

    # Convert to GeoJSON mapping, then to ee.Geometry
    return ee.Geometry(geom.__geo_interface__)

def _read_uploaded_vector(path: str) -> gpd.GeoDataFrame:
    """
    Read uploaded vector file into GeoDataFrame.
    Supports: GeoJSON/JSON, GPKG, KML/KMZ, Shapefile (via ZIP).
    """
    ext = os.path.splitext(path)[1].lower()

    if ext in [".geojson", ".json", ".gpkg", ".kml"]:
        return gpd.read_file(path)

    if ext == ".kmz":
        # KMZ is a zipped KML; geopandas can read it in some environments,
        # but often needs manual unzip.
        tmp_dir = os.path.join("/mnt/data", "aoi_kmz")
        os.makedirs(tmp_dir, exist_ok=True)
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(tmp_dir)
        # Find first .kml
        kml_path = None
        for root, _, files_ in os.walk(tmp_dir):
            for f in files_:
                if f.lower().endswith(".kml"):
                    kml_path = os.path.join(root, f)
                    break
            if kml_path:
                break
        if not kml_path:
            raise ValueError("KMZ does not contain a KML file.")
        return gpd.read_file(kml_path)

    if ext == ".zip":
        # Expect zipped Shapefile (.shp/.shx/.dbf and ideally .prj)
        tmp_dir = os.path.join("/mnt/data", "aoi_shp")
        os.makedirs(tmp_dir, exist_ok=True)
        with zipfile.ZipFile(path, "r") as z:
            z.extractall(tmp_dir)

        shp_path = None
        for root, _, files_ in os.walk(tmp_dir):
            for f in files_:
                if f.lower().endswith(".shp"):
                    shp_path = os.path.join(root, f)
                    break
            if shp_path:
                break

        if not shp_path:
            raise ValueError("ZIP does not contain a .shp file.")

        return gpd.read_file(shp_path)

    raise ValueError(f"Unsupported file type: {ext}")

@upload_btn.on_click
def _upload_aoi(_):
    global AOI_EE
    status.clear_output()

    with status:
        if mode_toggle.value != "upload":
            print("Switch AOI mode to 'Upload file' first.")
            return

        # Let user upload one file (GeoJSON / GPKG / KML/KMZ / ZIP)
        up = files.upload()
        if not up:
            print("No file uploaded.")
            return

        filename = list(up.keys())[0]
        path = os.path.join(os.getcwd(), filename)

        print(f"Reading AOI from: {filename}")
        gdf = _read_uploaded_vector(path)
        AOI_EE = _gdf_to_ee_geometry(gdf)

        # Visual feedback on the map
        Map.layers = Map.layers[:2]  # keep basemaps only
        Map.addLayer(AOI_EE, {"color": "yellow"}, "AOI (uploaded)")
        Map.centerObject(AOI_EE, 11)

        print("✅ AOI loaded and stored as AOI_EE (ee.Geometry).")
        print("You can now run the pipeline cell.")


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
#@title 💧 Run TWI Calculation { display-mode: "form" }

#@markdown ### 🔹 **How to use this step:**
#@markdown 1. **Run this cell** to display the parameter panel below.
#@markdown 2. Select:
#@markdown    - the **DEM source**,
#@markdown    - the **flow direction method**,
#@markdown    - and an optional **buffer size (km)** for accumulation.
#@markdown 3. Click **“Run TWI calculation”** to start processing.
#@markdown ---
#@markdown ### 🔹 **AOI input:**
#@markdown The Area of Interest (AOI) is taken in this priority:
#@markdown 1) **Uploaded AOI** from the AOI upload cell, otherwise
#@markdown 2) **Last geometry drawn on the map**
#@markdown ---
#@markdown ### 🔹 **What happens during execution:**
#@markdown 1. The **original AOI** is used for **final clipping** of all outputs.
#@markdown 2. For accumulation a **bbox + buffer** domain is created.
#@markdown 3. The workflow performs **DEM conditioning**, computes **flow direction**, **flow accumulation**, and **TWI**.
#@markdown 4. Outputs are cached for later export
#@markdown 5. In **cloud mode**, an interactive Earth Engine map is displayed.

import importlib, main
importlib.reload(main)

# Use the map from CELL 1; if missing, create one as a fallback.
if "Map" not in globals() or not isinstance(Map, geemap.Map):
    Map = geemap.Map(basemap="Esri.WorldImagery")
    Map.add_basemap("Esri.WorldTopoMap")
    display(Map)
    print("⚠️ Define an AOI first (draw on map or upload).")

# --- Controls ---
label_w = "140px"     # label width
w_left  = "540px"     # DEM dropdown width (longer)
w_right = "400px"     # Flow dropdown width (shorter)

DEM_OPTIONS = [
    ("FABDEM — 30 m — DTM (forests & buildings removed)", "FABDEM"),
    ("Copernicus GLO-30 — 30 m — DSM", "GLO30"),
    ("ALOS World 3D — 30 m — DSM", "AW3D30"),
    ("NASA SRTM — 30 m — DSM", "SRTMGL1_003"),
    ("NASADEM: NASA 30m — 30 m — DSM", "NASADEM_HGT"),
    ("ASTER GDEM — 30 m — DSM", "ASTER_GDEM"),
    ("SRTM — 90 m — DSM", "CGIAR_SRTM90"),
    ("MERIT DEM — 90 m — DTM", "MERIT_DEM"),
    ("MERIT Hydro (elv) — 90 m — DTM (hydrologically adjusted)", "MERIT_Hydro"),
]

FLOW_OPTIONS = [
    ("MFD (Quinn 1991)", "mfd_quinn_1991"),
    ("D8", "d8"),
]

def _make_dropdown(options, default_value, description, width):
    """Create a styled dropdown with a safe default value."""
    valid_values = [v if not isinstance(v, tuple) else v[1] for v in options]
    value = default_value if default_value in valid_values else valid_values[0]
    return widgets.Dropdown(
        options=options,
        value=value,
        description=description,
        style={"description_width": label_w},
        layout=widgets.Layout(width=width, margin="0 0 0 -35px"),
    )

dem_dropdown  = _make_dropdown(DEM_OPTIONS,  "FABDEM",     "DEM:",            w_left)
flow_dropdown = _make_dropdown(FLOW_OPTIONS, "quinn_1991", "Flow direction:", w_right)

# Small fixed spacer so the right dropdown is not glued to the edge
spacer = widgets.Box(layout=widgets.Layout(width="16px", flex="0 0 auto"))

row_top = widgets.HBox(
    [dem_dropdown, spacer, flow_dropdown],
    layout=widgets.Layout(width="100%", align_items="center")
)

buffer_km = widgets.FloatSlider(
    value=5.0, min=1.0, max=30.0, step=0.5,
    description="Buffer (km):", readout=True,
    style={"description_width": label_w},
    layout=widgets.Layout(width="520px", margin="4px 0 0 0")
)

# Run button + output area
run_btn = widgets.Button(
    description="Run TWI calculation",
    button_style="primary",
    tooltip="Run TWI calculation",
    layout=widgets.Layout(margin="6px 0 0 0")
)
out = widgets.Output()

controls = widgets.VBox([row_top, buffer_km, run_btn, out])
display(controls)

# Global holder for later export (no recompute)
PIPELINE_RES = {}

def _merge_drawn_geometry(m: geemap.Map):
    """Return a single ee.Geometry from user drawings (original AOI)."""
    roi = getattr(m, "user_roi", None)
    if roi is not None:
        return roi
    fc = getattr(m, "_user_rois", None)
    if fc is not None:
        try:
            return ee.FeatureCollection(fc).geometry()
        except Exception:
            return None
    return None

def _get_aoi_geometry(m: geemap.Map):
    """
    Return AOI ee.Geometry with priority:
    1) AOI_EE (uploaded AOI) if present
    2) last drawn geometry on the map
    """
    aoi = globals().get("AOI_EE", None)
    if aoi is not None:
        return aoi
    return _merge_drawn_geometry(m)

def _accum_geometry_from_roi(geom: ee.Geometry, buf_km: float) -> ee.Geometry:
    """Make bbox+buffer geometry for accumulation domain."""
    g = geom.bounds(1)  # ~1 m maxError
    if buf_km and buf_km > 0:
        g = g.buffer(buf_km * 1000)
    return g

@run_btn.on_click
def _run_pipeline(_):
    run_btn.disabled = True
    out.clear_output()

    with out:
        # 1) Collect original AOI (for final clipping)
        roi = _get_aoi_geometry(Map)
        if roi is None:
            print("⚠️ No AOI found. Draw an AOI on the map or upload a vector file first.")
            run_btn.disabled = False
            return

        # 2) Build accumulation domain (bbox + optional buffer)
        roi_acc = _accum_geometry_from_roi(roi, buffer_km.value)

        # 3) Run the pipeline with user-selected options
        print("▶ Running pipeline… (ACC on bbox+buffer, final clip on ORIGINAL AOI)")
        try:
            res = main.run_pipeline(
                project_id=GEE_PROJECT_ID,
                geometry=roi,            # ORIGINAL AOI → used for final clip
                accum_geometry=roi_acc,  # BBOX+BUFFER → used for accumulation domain
                dem_source=dem_dropdown.value,
                flow_method=flow_dropdown.value,
                use_bucket=USE_BUCKET,
            )
        except Exception as e:
            print("❌ Pipeline failed:", e)
            run_btn.disabled = False
            return

        # 4) Cache outputs for export
        print("✅ Done.")
        global PIPELINE_RES
        PIPELINE_RES = res

        mode = res.get("mode", None)

        if mode == "cloud":
            # Cloud mode → interactive EE map
            print("Displaying interactive TWI map from Earth Engine…")
            display(res["map"])

            try:
                Map.centerObject(roi)
                Map.addLayer(roi, {"color": "yellow"}, "Original AOI (final clip)")
                Map.addLayer(roi_acc, {"color": "orange"}, "Accumulation domain (bbox+buffer)")
            except Exception:
                pass

        elif mode == "local":
            # Local mode → static plots were already created inside run_pipeline()
            print("Local mode: outputs were computed and cached as GeoTIFF paths.")

    run_btn.disabled = False

In [9]:
#@title 📤 Export Layers (Google Drive or Local) { display-mode: "form" }

#@markdown ### 🔹 **This cell exports selected output layers from the completed TWI pipeline.**
#@markdown Choose **which layer to export** using the dropdown below (TWI, slope, flow accumulation, etc.).
#@markdown
#@markdown Export method depends on pipeline mode:
#@markdown - **Cloud mode:** server-side export to Google Drive (EE Tasks)
#@markdown - **Local mode:** download cached GeoTIFF from Colab
#@markdown
#@markdown ▶ **Note:** Run the pipeline first so outputs exist in `PIPELINE_RES`.

import ee
import os
import shutil
from google.colab import files
import ipywidgets as widgets
from IPython.display import display

# ------------------------------------------------------------
# Basic validation: pipeline must have been run before export
# ------------------------------------------------------------
if not globals().get("PIPELINE_RES"):
    raise RuntimeError("No cached outputs found. Run the pipeline first.")

mode = PIPELINE_RES.get("mode", "local")
print(f"Detected mode: {mode}")

# ------------------------------------------------------------
# Stable naming from UI selections (if available)
# ------------------------------------------------------------
dem_id = dem_dropdown.value if "dem_dropdown" in globals() else "DEM"
flow_id = flow_dropdown.value if "flow_dropdown" in globals() else "FLOW"

PREFIX_DEM_ONLY = f"_{dem_id}"            # e.g. FABDEM
PREFIX_DEM_FLOW = f"_{dem_id}_{flow_id}"  # e.g. FABDEM_qin_2007
WITHOUT_PREFIX = f""

# ------------------------------------------------------------
# Single layer registry (works for both cloud and local)
# Keys MUST match run_pipeline() return dict
# ------------------------------------------------------------
LAYER_REGISTRY = [
    ("TWI", "twi"),
    ("Slope", "slope"),
    ("Flow accumulation (km²)", "flow_accumulation_km2"),
    ("Flow accumulation (cells)", "flow_accumulation_cells"),
    # Optional extras:
    ("DEM", "dem"),
    #("Flow accumulation cells (full)", "flow_accumulation_cells_full"),
    #("Flow accumulation km² (full)", "flow_accumulation_km2_full"),
    ("(reference) MERIT flow accumulation upa (km²)", "MERIT_flow_accumulation_upa"),
    ("(reference) CTI - Hydrography90m", "cti_Hydrography90m")
]

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def _is_ee_image(x) -> bool:
    """Return True if object behaves like an EE Image."""
    return isinstance(x, ee.image.Image)

def _is_existing_path(x) -> bool:
    """Return True if x is a path to an existing file."""
    return isinstance(x, str) and os.path.exists(x)

def _is_exportable(key: str) -> bool:
    """Check if given layer key exists and is exportable in the current mode."""
    v = PIPELINE_RES.get(key)
    if mode == "cloud":
        return _is_ee_image(v)
    return _is_existing_path(v)

def _prefix(layer_key: str) -> str:
    """
    Return a naming suffix based on layer dependency:
    - Flow-dependent layers: DEM + FLOW
    - DEM-dependent layers: DEM only
    """
    flow_dependent = {
        "twi",
        "flow_accumulation_km2",
        "flow_accumulation_cells",
        "twi_full",
        "flow_accumulation_km2_full",
        "flow_accumulation_cells_full",
    }
    dem_dependent = {
        "slope",
        "dem_full",
        "slope_full",
    }

    if layer_key in flow_dependent:
        return PREFIX_DEM_FLOW
    if layer_key in dem_dependent:
        return PREFIX_DEM_ONLY
    return WITHOUT_PREFIX

def _copy_with_name(src_path: str, out_name_no_ext: str) -> str:
    """
    Copy source GeoTIFF to a target filename in the same directory.
    Returns the final path. If copy fails, returns original path.
    """
    base_dir = os.path.dirname(src_path)
    out_path = os.path.join(base_dir, out_name_no_ext + ".tif")

    if os.path.abspath(out_path) == os.path.abspath(src_path):
        return src_path

    try:
        shutil.copy2(src_path, out_path)
        return out_path
    except Exception:
        return src_path

# ------------------------------------------------------------
# Build dropdown options based on what is actually available
# ------------------------------------------------------------
options = [(name, key) for (name, key) in LAYER_REGISTRY if _is_exportable(key)]
if not options:
    raise RuntimeError("No exportable layers found in PIPELINE_RES for this mode.")

# ------------------------------------------------------------
# UI widgets
# ------------------------------------------------------------
layer_dd = widgets.Dropdown(
    options=options,
    value=options[0][1],
    description="Layer:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width="560px", margin="0 0 0 -20px"),
)

drive_folder = widgets.Text(
    value="GEE_Exports",
    description="Drive folder:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width="560px", margin="0 0 0 -20px"),
)

export_btn = widgets.Button(
    description="Export / Download",
    button_style="primary",
    layout=widgets.Layout(margin="8px 0 0 0"),
)

# IMPORTANT: output must be unique to this cell, otherwise it can “jump” under another cell
EXPORT_OUT = widgets.Output()

ui = widgets.VBox([layer_dd] + ([drive_folder] if mode == "cloud" else []) + [export_btn, EXPORT_OUT])
display(ui)

# ------------------------------------------------------------
# Export logic
# ------------------------------------------------------------
@export_btn.on_click
def _do_export(_):
    export_btn.disabled = True
    EXPORT_OUT.clear_output()

    with EXPORT_OUT:
        layer_key = layer_dd.value
        out_name = f"{layer_key}{_prefix(layer_key)}".replace(" ", "_")

        if mode == "cloud":
            img = PIPELINE_RES.get(layer_key)
            if not _is_ee_image(img):
                raise TypeError(f"Selected layer is not an ee.Image: {layer_key}")

            region = PIPELINE_RES["geometry"]
            scale = PIPELINE_RES["scale"]
            nodata_val = -9999.0

            ee_img = ee.Image(img).select(0).toFloat().unmask(nodata_val)

            task = ee.batch.Export.image.toDrive(
                image=ee_img,
                description=f"Export_{out_name}",
                folder=drive_folder.value,
                fileNamePrefix=out_name,
                region=region,
                scale=scale,
                maxPixels=1e13,
                fileFormat="GeoTIFF",
                formatOptions={"noData": nodata_val},
            )
            task.start()

            print("📤 Export started in Earth Engine Tasks.")
            print(f"Layer: {layer_key}")
            print(f"Drive folder: {drive_folder.value}")
            print(f"File prefix: {out_name}")

        else:
            src_path = PIPELINE_RES.get(layer_key)
            if not _is_existing_path(src_path):
                raise FileNotFoundError(f"Selected file not found: {src_path}")

            final_path = _copy_with_name(src_path, out_name)

            print("⬇️ Preparing local download:")
            print(f"Layer: {layer_key}")
            print(f"File path: {final_path}")

            try:
                files.download(final_path)
            except Exception as e:
                print(f"⚠️ Auto-download failed: {e}")
                print("You can still download the file from the left ‘Files’ panel.")

    export_btn.disabled = False

Detected mode: cloud
